In [ ]:
#  Ensure that all necessary Python libraries (listed below) are downloaded

import numpy as np
import pandas as pd
import os
import datetime
import xml.etree.ElementTree as ET
import holidays
import tkinter.filedialog
import astral
from astral import sun
from astral.geocoder import database

In [ ]:
root = tkinter.Tk()
request_file = tkinter.filedialog.askopenfilename(initialdir=os.getcwd(),filetypes=[('XML Files', '*.xml')])
root.destroy()
xml_file_name = request_file.split('/')[-1]
xml_file_name

In [ ]:
annual_tides = ET.parse(xml_file_name).getroot()

In [ ]:
station = annual_tides[4].text
station

In [ ]:
annual_tides[5].text

In [ ]:
tides_year = annual_tides[8].text[:4]
tides_year

In [ ]:
# look up latitude and longitude using "https://www.ngs.noaa.gov/NCAT" this will only effect the sunrise and sunset variables

loc = astral.LocationInfo(name=station, 
                    region='USA', 
                    timezone='US/Pacific', 
                    latitude=36.6013000166, 
                    longitude=-121.8824529648)

In [ ]:
col_names = list()
for n in range(0,6):
    col_names.append(annual_tides[-1][0][n].tag)
col_names.remove('date')
col_names.remove('time')
col_names.insert(0,'date_time')
col_names

In [ ]:
f = annual_tides.find(annual_tides[-1].tag)
df_tides = pd.DataFrame(index=range(0,len(list(f.iter(annual_tides[-1][0].tag)))),columns=col_names)
df_tides

In [ ]:
for i in df_tides.index:
    df_tides['date_time'].iloc[i]=datetime.datetime.strptime(annual_tides[-1][i][0].text+annual_tides[-1][i][2].text,'%Y/%m/%d%I:%M %p')
    df_tides[annual_tides[-1][0][1].tag].iloc[i]=annual_tides[-1][i][1].text
    df_tides[annual_tides[-1][0][3].tag].iloc[i]=float(annual_tides[-1][i][3].text)
    df_tides[annual_tides[-1][0][4].tag].iloc[i]=float(annual_tides[-1][i][4].text)
    df_tides[annual_tides[-1][0][5].tag].iloc[i]=annual_tides[-1][i][5].text
df_tides

In [ ]:
df_copy = df_tides.copy()
for i in df_tides.index:
    neg_bool = df_tides['pred_in_cm'][i]<=0
    off_day_bool = df_tides['day'][i] == 'Sat' or df_tides['day'][i] =='Sun' or df_tides['date_time'][i].date() in holidays.US(years=int(tides_year))
    sr = astral.sun.sun(loc.observer,tzinfo='US/Pacific',date=df_tides['date_time'][i].date())['sunrise']
    ss = astral.sun.sun(loc.observer,tzinfo='US/Pacific',date=df_tides['date_time'][i].date())['sunset']
    after_work_bool = datetime.time(16,0,0,0) <= df_tides['date_time'][i].time() < ss.time()
    
    # Use either of these daylight_bool variables. The first gives sunrise to sunset, the second gives 15 mins before sunrise to 10 mins before sunset
    
    #daylight_bool = sr.time() <= df_tides['date_time'][i].time() < ss.time()
    daylight_bool = (sr-datetime.timedelta(minutes=15)).time()<=df_tides['date_time'][i].time()<(ss-datetime.timedelta(minutes=10)).time()
    
    if (not neg_bool or not daylight_bool) or (not off_day_bool and not after_work_bool):
        df_copy.drop(index=i,inplace=True)
    else:
        pass
df_copy

In [ ]:
df_goog_cal = pd.DataFrame(columns=['Subject','Start Date','Start Time','End Date','End Time','All Day Event','Private','Description','Location'],index=df_copy.index)
for i in df_goog_cal.index:
    df_goog_cal['Subject'].loc[i] = 'Low Tide'
    df_goog_cal['Start Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['Start Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['End Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['End Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['All Day Event'].loc[i] = False
    df_goog_cal['Private'].loc[i] = False
    low_water_mark = df_copy['pred_in_ft'][i]
    df_goog_cal['Description'].loc[i] = f'Low water mark at {low_water_mark} ft'
    df_goog_cal['Location'].loc[i] = f'{annual_tides[4].text}, {annual_tides[5].text}'
df_goog_cal

In [ ]:
print(f'There are a total of {len(df_goog_cal)} convenient low tide events for the year {tides_year}')

In [ ]:
df_goog_cal.to_csv(f'{station}_lowest_tides{tides_year}.csv')